In [1]:
#Colab setting directory
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/Porsche_interview') #Change the directory
os.listdir() #list the directory

Mounted at /content/drive


['Dataset',
 'best_weight.pt',
 'saved_val_acc.txt',
 'runs',
 'models',
 'Untitled0.ipynb',
 'Porsche_Interview.ipynb']

In [ ]:
!wget -P Dataset/ https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
!tar -xzvf {Dataset_root + '/cifar-100-python.tar.gz'}
!rm {Dataset_root + '/cifar-100-python.tar.gz'}

--2022-08-11 09:31:44--  https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169001437 (161M) [application/x-gzip]
Saving to: ‘Dataset/cifar-100-python.tar.gz’

cifar-100-python.ta 100%[===================>] 161.17M  57.5MB/s    in 2.8s    

2022-08-11 09:31:49 (57.5 MB/s) - ‘Dataset/cifar-100-python.tar.gz’ saved [169001437/169001437]

cifar-100-python/
cifar-100-python/file.txt~
cifar-100-python/train
cifar-100-python/test
cifar-100-python/meta


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from torchsummary import summary
from torch.optim import lr_scheduler

In [3]:
Dataset_Dir = "./Dataset"
input_size = 224

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])
}

# image_datasets = {x: datasets.ImageFolder(os.path.join(Dataset_Dir, x), data_transforms[x]) for x in ['train', 'val']}
# image_datasets = {x: datasets.ImageFolder(os.path.join(Dataset_Dir, x), data_transforms[x]) for x in ['train', 'val']}

# dataloader_dict = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}
# image_datasets = datasets.CIFAR100(Dataset_Dir, train=True, transform=data_transforms['train'])

train_set = datasets.CIFAR100(Dataset_Dir, train=True, transform=data_transforms['train'])
val_set = datasets.CIFAR100(Dataset_Dir, train=False, transform=data_transforms['val'])
data_loader = {'train': DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2), 'val': DataLoader(val_set, batch_size=32, shuffle=False, num_workers=2)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
def initialize_model(model_name, num_classes, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained, progress=True)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
    elif model_name == "wrn":
        model_ft = models.wide_resnet50_2(pretrained=use_pretrained, progress=True)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
    elif model_name == "mobile":
        model_ft = models.mobilenet_v3_large(pretrained=use_pretrained, progress=True)
        num_ftrs = model_ft.classifier[3].in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
    elif model_name == "resnext":
        model_ft = models.resnext50_32x4d(pretrained=use_pretrained, progress=True)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
    
    return model_ft

# model_name = "resnet"
# model_name = "mobile"
model_name = "resnext"
num_classes = 100
use_pretrained = False


# Initialize the model for this run
model_ft = initialize_model(model_name, num_classes, use_pretrained)

# Print the model we just instantiated

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [23]:
datetime.now()
a = "./models/" + str(datetime.now()) + "/best_weight.pt"
a

'./models/2022-08-13 13:31:33.933332/best_weight.pt'

In [6]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, from_epoch=0):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    weight_folder = "./models/" + str(datetime.now())
    os.makedirs(weight_folder)
    writer = SummaryWriter()
    # scheduler = lr_scheduler.LinearLR(optimizer, 0.5, 1, 30)
    for epoch in range(from_epoch, num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))


            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model, weight_folder+'/best_weight.pt')

            if phase == 'train':
                train_loss = epoch_loss
                train_acc = epoch_acc
            else:
                val_loss = epoch_loss
                val_acc = epoch_acc

        writer.add_scalars('loss', {
            'train_loss': train_loss,
            'val_loss': val_loss
        }, epoch+1)
        writer.add_scalars('accuracy', {
            'train_accuracy': train_acc,
            'val_accuracy': val_acc            
        }, epoch+1)
        
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
scratch_model, _ = initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=False)
scartch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
num_epochs = 100
_,scratch_hist = train_model(scratch_model, data_loader, scratch_criterion, scratch_optimizer, num_epochs=num_epochs, is_inception=(model_name=="inception"))

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/100
----------
train Loss: 4.0665 Acc: 0.0799
val Loss: 3.6619 Acc: 0.1379

Epoch 2/100
----------
train Loss: 3.7117 Acc: 0.1348
val Loss: 3.3419 Acc: 0.1969

Epoch 3/100
----------
train Loss: 3.5002 Acc: 0.1689
val Loss: 3.1087 Acc: 0.2399

Epoch 4/100
----------
train Loss: 3.3130 Acc: 0.2013
val Loss: 2.9492 Acc: 0.2725

Epoch 5/100
----------
train Loss: 3.1493 Acc: 0.2308
val Loss: 2.7907 Acc: 0.2996

Epoch 6/100
----------
train Loss: 2.9945 Acc: 0.2611
val Loss: 2.6113 Acc: 0.3382

Epoch 7/100
----------
train Loss: 2.8693 Acc: 0.2888
val Loss: 2.6449 Acc: 0.3346

Epoch 8/100
----------
train Loss: 2.7554 Acc: 0.3111
val Loss: 2.3170 Acc: 0.3970

Epoch 9/100
----------
train Loss: 2.6353 Acc: 0.3354
val Loss: 2.2131 Acc: 0.4186

Epoch 10/100
----------
train Loss: 2.5459 Acc: 0.3542
val Loss: 2.1460 Acc: 0.4341

Epoch 11/100
----------
train Loss: 2.4660 Acc: 0.3697
val Loss: 2.0810 Acc: 0.4481

Epoch 12/100
----------
train Loss: 2.3556 Acc: 0.3928
val Loss: 1.9418 Ac

In [ ]:
scratch_model = initialize_model(model_name, num_classes, use_pretrained=False)
scartch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
num_epochs = 100
_,scratch_hist = train_model(scratch_model, data_loader, scratch_criterion, scratch_optimizer, num_epochs=num_epochs)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/100
----------
train Loss: 4.0548 Acc: 0.0819
val Loss: 3.7310 Acc: 0.1359

Epoch 2/100
----------
train Loss: 3.7219 Acc: 0.1356
val Loss: 3.3633 Acc: 0.1933

Epoch 3/100
----------
train Loss: 3.5099 Acc: 0.1694
val Loss: 3.1361 Acc: 0.2378

Epoch 4/100
----------
train Loss: 3.3324 Acc: 0.1994
val Loss: 2.9173 Acc: 0.2767

Epoch 5/100
----------
train Loss: 3.1554 Acc: 0.2325
val Loss: 2.7365 Acc: 0.3091

Epoch 6/100
----------
train Loss: 3.0109 Acc: 0.2572
val Loss: 2.6117 Acc: 0.3340

Epoch 7/100
----------
train Loss: 2.8790 Acc: 0.2855
val Loss: 2.4562 Acc: 0.3714

Epoch 8/100
----------
train Loss: 2.7593 Acc: 0.3106
val Loss: 2.3348 Acc: 0.3929

Epoch 9/100
----------
train Loss: 2.6535 Acc: 0.3325
val Loss: 2.2171 Acc: 0.4211

Epoch 10/100
----------
train Loss: 2.5542 Acc: 0.3525
val Loss: 2.2216 Acc: 0.4278

Epoch 11/100
----------
train Loss: 2.4739 Acc: 0.3696
val Loss: 2.0549 Acc: 0.4626

Epoch 12/100
----------
train Loss: 2.3938 Acc: 0.3857
val Loss: 2.0273 Ac

In [11]:
continued_model = initialize_model(model_name, num_classes, use_pretrained=False)
continued_model.load_state_dict(torch.load('best_weight.pt').state_dict())
continued_model = continued_model.to(device)
continued_optimizer = optim.SGD(continued_model.parameters(), lr=0.001, momentum=0.9)
continued_criterion = nn.CrossEntropyLoss()
num_epochs = 100
model ,continued_hist = train_model(continued_model, data_loader, continued_criterion, continued_optimizer, num_epochs=num_epochs, from_epoch=59)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


FileNotFoundError: ignored

In [ ]:
test_model = initialize_model(model_name, num_classes, use_pretrained=False)
test_model.load_state_dict(torch.load('best_weight.pt').state_dict())

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [20]:
continued_model = initialize_model(model_name, num_classes, use_pretrained=False)
continued_model.load_state_dict(torch.load('best_weight.pt').state_dict())
continued_model = continued_model.to(device)
continued_optimizer = optim.SGD(continued_model.parameters(), lr=0.0001, momentum=0.9)
continued_criterion = nn.CrossEntropyLoss()
num_epochs = 200
model ,continued_hist = train_model(continued_model, data_loader, continued_criterion, continued_optimizer, num_epochs=num_epochs, from_epoch=100)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 101/200
----------
train Loss: 0.7818 Acc: 0.7888
val Loss: 1.1036 Acc: 0.7271

Epoch 102/200
----------
train Loss: 0.7323 Acc: 0.8038
val Loss: 1.0855 Acc: 0.7278

Epoch 103/200
----------
train Loss: 0.7250 Acc: 0.8062
val Loss: 1.0931 Acc: 0.7283

Epoch 104/200
----------
train Loss: 0.7203 Acc: 0.8056
val Loss: 1.0850 Acc: 0.7321

Epoch 105/200
----------
train Loss: 0.7148 Acc: 0.8077
val Loss: 1.0853 Acc: 0.7291

Epoch 106/200
----------
train Loss: 0.7093 Acc: 0.8085
val Loss: 1.0768 Acc: 0.7311

Epoch 107/200
----------
train Loss: 0.6901 Acc: 0.8157
val Loss: 1.0898 Acc: 0.7310

Epoch 108/200
----------
train Loss: 0.6971 Acc: 0.8146
val Loss: 1.0925 Acc: 0.7289

Epoch 109/200
----------
train Loss: 0.6874 Acc: 0.8154
val Loss: 1.0795 Acc: 0.7338

Epoch 110/200
----------
train Loss: 0.6951 Acc: 0.8139
val Loss: 1.0844 Acc: 0.7270

Epoch 111/200
----------
train Loss: 0.6857 Acc: 0.8175
val Loss: 1.0861 Acc: 0.7294

Epoch 112/200
----------
train Loss: 0.6718 Acc: 0.819

KeyboardInterrupt: ignored

In [ ]:
scratch_model = initialize_model(model_name, num_classes, use_pretrained=False)
scartch_model = scratch_model.to(device)
scratch_optimizer = optim.Adam(scratch_model.parameters(), lr=0.001)
scratch_criterion = nn.CrossEntropyLoss()
num_epochs = 100
_,scratch_hist = train_model(scratch_model, data_loader, scratch_criterion, scratch_optimizer, num_epochs=num_epochs)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/100
----------
train Loss: 4.2111 Acc: 0.0588
val Loss: 3.8980 Acc: 0.0987

Epoch 2/100
----------
train Loss: 3.8680 Acc: 0.1066
val Loss: 3.5151 Acc: 0.1650

Epoch 3/100
----------
train Loss: 3.5631 Acc: 0.1561
val Loss: 3.9060 Acc: 0.2070

Epoch 4/100
----------
train Loss: 3.2900 Acc: 0.2034
val Loss: 3.3332 Acc: 0.2687

Epoch 5/100
----------
train Loss: 3.0730 Acc: 0.2464
val Loss: 2.7281 Acc: 0.3078

Epoch 6/100
----------
train Loss: 2.8939 Acc: 0.2818
val Loss: 2.5806 Acc: 0.3449

Epoch 7/100
----------
train Loss: 2.7359 Acc: 0.3126
val Loss: 2.3918 Acc: 0.3794

Epoch 8/100
----------
train Loss: 2.6379 Acc: 0.3343
val Loss: 3.3850 Acc: 0.3917

Epoch 9/100
----------
train Loss: 2.5333 Acc: 0.3566
val Loss: 2.4360 Acc: 0.3861

Epoch 10/100
----------
train Loss: 2.4589 Acc: 0.3743
val Loss: 2.1706 Acc: 0.4468

Epoch 11/100
----------


In [13]:
continued_model = initialize_model(model_name, num_classes, use_pretrained=False)
continued_model.load_state_dict(torch.load('./models/2022-08-13 17:18:09.166664/best_weight.pt').state_dict())
continued_model = continued_model.to(device)
continued_optimizer = optim.SGD(continued_model.parameters(), lr=0.001, momentum=0.9)
continued_criterion = nn.CrossEntropyLoss()
num_epochs = 50
model ,continued_hist = train_model(continued_model, data_loader, continued_criterion, continued_optimizer, num_epochs=num_epochs, from_epoch=10)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 11/50
----------
train Loss: 2.2868 Acc: 0.4152
val Loss: 2.1966 Acc: 0.4709

Epoch 12/50
----------
train Loss: 2.2429 Acc: 0.4238
val Loss: 2.2699 Acc: 0.4721

Epoch 13/50
----------
train Loss: 2.2215 Acc: 0.4300
val Loss: 2.0536 Acc: 0.4805

Epoch 14/50
----------
train Loss: 2.2090 Acc: 0.4310
val Loss: 2.0261 Acc: 0.4828

Epoch 15/50
----------
train Loss: 2.1912 Acc: 0.4366
val Loss: 2.2163 Acc: 0.4846

Epoch 16/50
----------
train Loss: 2.1926 Acc: 0.4351
val Loss: 2.2522 Acc: 0.4842

Epoch 17/50
----------
train Loss: 2.1701 Acc: 0.4386
val Loss: 2.1196 Acc: 0.4873

Epoch 18/50
----------
train Loss: 2.1666 Acc: 0.4384
val Loss: 2.1146 Acc: 0.4896

Epoch 19/50
----------


KeyboardInterrupt: ignored

In [ ]:
scratch_model = initialize_model(model_name, num_classes, use_pretrained=False)
scartch_model = scratch_model.to(device)
scratch_optimizer = optim.Adam(scratch_model.parameters(), lr=0.001)
scratch_criterion = nn.CrossEntropyLoss()
num_epochs = 30
_,scratch_hist = train_model(scratch_model, data_loader, scratch_criterion, scratch_optimizer, num_epochs=num_epochs)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/30
----------
train Loss: 4.1211 Acc: 0.0710
val Loss: 4.5368 Acc: 0.0988

Epoch 2/30
----------
train Loss: 3.6954 Acc: 0.1331
val Loss: 3.6141 Acc: 0.1585

Epoch 3/30
----------
train Loss: 3.3870 Acc: 0.1858
val Loss: 3.1569 Acc: 0.2194

Epoch 4/30
----------
train Loss: 3.0752 Acc: 0.2418
val Loss: 3.1923 Acc: 0.3070

Epoch 5/30
----------
train Loss: 2.7892 Acc: 0.2974
val Loss: 2.9240 Acc: 0.3599

Epoch 6/30
----------
train Loss: 2.5653 Acc: 0.3430
val Loss: 2.3477 Acc: 0.3796

Epoch 7/30
----------
train Loss: 2.3955 Acc: 0.3834
val Loss: 2.0680 Acc: 0.4496

Epoch 8/30
----------
train Loss: 2.2315 Acc: 0.4176
val Loss: 1.8526 Acc: 0.4972

Epoch 9/30
----------
train Loss: 2.1050 Acc: 0.4453
val Loss: 1.8564 Acc: 0.5013

Epoch 10/30
----------
train Loss: 2.0025 Acc: 0.4707
val Loss: 1.7109 Acc: 0.5341

Epoch 11/30
----------
train Loss: 1.9078 Acc: 0.4933
val Loss: 1.5921 Acc: 0.5578

Epoch 12/30
----------
train Loss: 1.8369 Acc: 0.5101
val Loss: 1.5673 Acc: 0.5685

E

In [ ]:
# resnext
continued_model = initialize_model(model_name, num_classes, use_pretrained=False)
continued_model.load_state_dict(torch.load('./models/2022-08-14 16:21:48.073675/best_weight.pt').state_dict())
continued_model = continued_model.to(device)
continued_optimizer = optim.Adam(continued_model.parameters(), lr=0.001)
continued_criterion = nn.CrossEntropyLoss()
num_epochs = 70
model ,continued_hist = train_model(continued_model, data_loader, continued_criterion, continued_optimizer, num_epochs=num_epochs, from_epoch=16)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 17/70
----------
train Loss: 1.6961 Acc: 0.5403
val Loss: 1.3923 Acc: 0.6158

Epoch 18/70
----------
train Loss: 1.6493 Acc: 0.5570
val Loss: 1.4110 Acc: 0.6128

Epoch 19/70
----------
train Loss: 1.5827 Acc: 0.5738
val Loss: 1.4466 Acc: 0.6243

Epoch 20/70
----------
train Loss: 1.5139 Acc: 0.5862
val Loss: 1.2866 Acc: 0.6448

Epoch 21/70
----------
train Loss: 1.4716 Acc: 0.5996
val Loss: 1.3010 Acc: 0.6413

Epoch 22/70
----------
train Loss: 1.4310 Acc: 0.6085
val Loss: 1.4090 Acc: 0.6364

Epoch 23/70
----------
train Loss: 1.3765 Acc: 0.6239
val Loss: 1.2440 Acc: 0.6562

Epoch 24/70
----------
train Loss: 1.3328 Acc: 0.6345
val Loss: 1.2198 Acc: 0.6648

Epoch 25/70
----------
train Loss: 1.2976 Acc: 0.6430
val Loss: 1.2375 Acc: 0.6607

Epoch 26/70
----------
train Loss: 1.2464 Acc: 0.6571
val Loss: 1.1812 Acc: 0.6813

Epoch 27/70
----------
train Loss: 1.2057 Acc: 0.6648
val Loss: 5.5659 Acc: 0.5657

Epoch 28/70
----------
train Loss: 1.1779 Acc: 0.6741
val Loss: 1.6104 Acc: 

In [8]:
# resnext
continued_model = initialize_model(model_name, num_classes, use_pretrained=False)
continued_model.load_state_dict(torch.load('./models/2022-08-15 16:26:33.793275/best_weight.pt', map_location=torch.device('cpu')).state_dict())
continued_model = continued_model.to(device)
continued_optimizer = optim.Adam(continued_model.parameters(), lr=0.0005)
continued_criterion = nn.CrossEntropyLoss()
num_epochs = 70
model ,continued_hist = train_model(continued_model, data_loader, continued_criterion, continued_optimizer, num_epochs=num_epochs, from_epoch=30)

Epoch 31/70
----------


KeyboardInterrupt: ignored